In [5]:
!pip install talos

    100% |████████████████████████████████| 51kB 17.4MB/s 
    100% |████████████████████████████████| 778kB 22.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/fc/c0/0e/4c615906c2fb59915976e570b068b90c8d973d904bc71f7773
  Stored in directory: /root/.cache/pip/wheels/e1/18/8a/dce70e6d93d855d97d435d3064122e44a375930bf7bffb8e97
  Stored in directory: /root/.cache/pip/wheels/a0/5c/ff/66b7352ff7b8b2b857101c734f2b8e7dc0b6840829951c8c87
  Stored in directory: /root/.cache/pip/wheels/36/6b/4c/e1fc6d7d8811940fbea1147b1519c7baa6933e4baeff904433
  Stored in directory: /root/.cache/pip/wheels/ce/ea/67/c7756f897f70f22279c7f2bbf5be52423d85835643042fcc34
Successfully built talos astetik chances kerasplotlib wrangle


In [6]:
import talos as ta
import keras
import time
import keras.backend as K
from keras.layers.core import Activation
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Conv2D, Flatten,Conv1D,Reshape

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn import preprocessing

# Setting seed for reproducibility
np.random.seed(1234)  
PYTHONHASHSEED = 0

# define path to save model
model_path = 'cnnid_regression_model.h5'

/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
Using TensorFlow backend.


Data Ingestion

In [ ]:
# read training data - It is the aircraft engine run-to-failure data.
train_df = pd.read_csv('train_FD003.txt', sep=" ", header=None)
test_df = pd.read_csv('test_FD003.txt', sep=" ", header=None)
truth_df = pd.read_csv('RUL_FD003.txt', sep=" ", header=None)
# pick a window size of 50 cycles
sequence_length = 50


In [ ]:
# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    # for one id I put all the rows in a single matrix
    data_matrix = id_df[seq_cols].values
    num_elements = data_matrix.shape[0]
    # Iterate over two lists in parallel.
    # For example id1 have 192 rows and sequence_length is equal to 50
    # so zip iterate over two following list of numbers (0,112),(50,192)
    # 0 50 -> from row 0 to row 50
    # 1 51 -> from row 1 to row 51
    # 2 52 -> from row 2 to row 52
    # ...
    # 111 191 -> from row 111 to 191
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_matrix[start:stop, :]

In [ ]:
# function to generate labels
def gen_labels(id_df, seq_length, label):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    # For one id I put all the labels in a single matrix.
    # For example:
    # [[1]
    # [4]
    # [1]
    # [5]
    # [9]
    # ...
    # [200]] 
    data_matrix = id_df[label].values
    num_elements = data_matrix.shape[0]
    # I have to remove the first seq_length labels
    # because for one id the first sequence of seq_length size have as target
    # the last label (the previus ones are discarded).
    # All the next id's sequences will have associated step by step one label as target.
    return data_matrix[(seq_length-1):(num_elements-1), :]

In [ ]:
# Using the scoring function

def scoring_function(true, pred):
    d = pred - true
    length = len(d)
    s = 0
    for i in range(length):
        if (d[i] < 0):
            s += np.exp((-d[i]/10))-1
        else:
            s += np.exp((d[i]/13))-1
    return s
def rmse(true, pred):
    d_squared = (pred - true)**2
    length = len(d_squared)
    err = 0
    for i in range(length):
        err += d_squared[i]
    err = np.sqrt(err/length)
    return err
def merror(true,pred):
    d=np.abs(pred-true)
    length = len(d)
    err=np.sum(d)/length
    return err
    
    

In [ ]:
#Preprocessing
def prep_deeplearn(train_df,test_df,truth_df,sequence_length):
    #column names
    col_names=['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']
    #remove the columns 26 and 27 because of NAN values
    train_df=train_df.loc[:,0:25]
    test_df=test_df.loc[:,0:25]
    
    train_df.columns = col_names
    test_df.columns=col_names
    train_df = train_df.sort_values(['id','cycle'])
    test_df = test_df.sort_values(['id','cycle'])
    
    #remove the columns 1 because of NAN values
    truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)
    truth_df.columns = ['cycle']
    
    #Data Preprocessing
    #Training dataset
    rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
    rul.columns = ['id', 'max']
    
    train_df = train_df.merge(rul, on=['id'], how='left')
    train_df['RUL'] = train_df['max'] - train_df['cycle']
    train_df.drop('max', axis=1, inplace=True)
    
    #check columns with repeated values: setting3, s1, s5, s10, s16,s18,s19
    
    train_df.drop(train_df.columns[[4,5,9,14,20,22,23]], axis=1, inplace=True)    
    
    test_df.drop(test_df.columns[[4,5,9,14,20,22,23]], axis=1, inplace=True)
    

    # We use the ground truth dataset to generate labels for the test data.
    # generate column max for test data
    rul = pd.DataFrame(test_df.groupby('id')['cycle'].max()).reset_index()
    rul.columns = ['id', 'max']
    
    truth_df.columns = ['more']
    truth_df['id'] = truth_df.index + 1
    truth_df['max'] = rul['max'] + truth_df['more']
    truth_df.drop('more', axis=1, inplace=True)
    
    # generate RUL for test data
    test_df = test_df.merge(truth_df, on=['id'], how='left')
    test_df['RUL'] = test_df['max'] - test_df['cycle']
    test_df.drop('max', axis=1, inplace=True)
    
    # MinMax normalization (from 0 to 1)
    min_max_scaler = preprocessing.MinMaxScaler()
    ###TRAIN#####
    train_df['cycle_norm'] = train_df['cycle']
    cols_normalize = train_df.columns.difference(['id','cycle','RUL','label1','label2'])
    norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                                 columns=cols_normalize, 
                                 index=train_df.index)
    join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
    train_df = join_df.reindex(columns = train_df.columns)
    ####TEST#####
    test_df['cycle_norm'] = test_df['cycle']
    norm_test_df = pd.DataFrame(min_max_scaler.transform(test_df[cols_normalize]), 
                                columns=cols_normalize, 
                                index=test_df.index)
    test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
    test_df = test_join_df.reindex(columns = test_df.columns)
    test_df = test_df.reset_index(drop=True)
    test_df.head(2)
    
    
    # pick the feature columns 
    sequence_cols = train_df.columns.difference(['id','cycle','RUL'])
    
    seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) 
           for id in train_df['id'].unique() if train_df[train_df['id']==id]['cycle'].max()>sequence_length)
    
    seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
    
    # transform each id of the test dataset in a sequence
    seq_test_gen = (list(gen_sequence(test_df[test_df['id']==id], sequence_length, sequence_cols))
           for id in test_df['id'].unique() if test_df[test_df['id']==id]['cycle'].max()>sequence_length)
    
    seq_test_array= np.concatenate(list(seq_test_gen)).astype(np.float32)
    
    label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['RUL']) 
             for id in train_df['id'].unique() if train_df[train_df['id']==id]['cycle'].max()>sequence_length]
    
    #merge labels to form an array of all labels
    label_array = np.concatenate(label_gen).astype(np.float32)
    
    return seq_array,seq_test_array,label_array,test_df




    

CNN architecture

In [ ]:
def minimal():
    p = {'activation':['relu', 'elu', 'tanh'],
         'optimizer': ['Nadam', 'Adam'],
         'losses': ['logcosh','mean_squared_error'],
         'kernel':[3,5,10],
         'batch_size': [16,32,128,512],
         'hidden_layers': [50,100,200],
         'dropout': [0.2,0.5]
        }
    
    def cnn1d_train(seq_array,label_array,x_val, y_val,params):
        nb_in=seq_array.shape[0]
        nb_features = seq_array.shape[2]
        nb_out = label_array.shape[1]
        model = Sequential()

        #model.add(Reshape((sequence_length, nb_features)))
        model.add(Conv1D(10,kernel_size=params['kernel'],activation=params['activation'],
                           data_format = 'channels_last',
                           input_shape = (sequence_length, nb_features)))

        model.add(Conv1D(10,kernel_size=params['kernel'], activation=params['activation']))

        model.add(Conv1D(10,kernel_size=params['kernel'], activation=params['activation']))

        model.add(Conv1D(10,kernel_size=params['kernel'], activation=params['activation']))

        model.add(Conv1D(10,kernel_size=params['kernel'], activation=params['activation']))

        model.add(Flatten())
        model.add(Dropout(rate=params['dropout']))
        model.add(Dense(units=params['hidden_layers'], activation=params['activation']))
        model.add(Dense(nb_out, activation=params['activation']))

        model.compile(loss=params['losses'], optimizer=params['optimizer'],metrics=['mae','acc'])

        start=time.time()
        history = model.fit(seq_array, label_array, epochs=100, batch_size=params['batch_size'], validation_split=0.1, verbose=2,
                         callbacks = [keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=15, verbose=0, mode='max'),
                                      keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='min', verbose=0)])
                  
        end=time.time()
        run_time=end-start

        return history,model
     def cnn2d_train(seq_array,label_array,x_val, y_val,params):
        # Expand training data to picture format with 1 representing greyscale picture
        seq_array_expanded = seq_array.reshape(nb_in, sequence_length, nb_features, 1)
        nb_in=seq_array.shape[0]
        nb_features = seq_array.shape[2]
        nb_out = label_array.shape[1]
        model = Sequential()

        #model.add(Reshape((sequence_length, nb_features)))
        model.add(Conv2D(filters=10, kernel_size=params['kernel'], strides=(1,1),
                 padding='same',
                 activation=params['activation'],
                 data_format = 'channels_last',
                 input_shape = (sequence_length, nb_features, 1)))
        model.add(Conv2D(filters=10, kernel_size=params['kernel'], strides=(1,1),
                         activation=params['activation'],
                         padding='same'))
        model.add(Conv2D(filters=10, kernel_size=params['kernel'], strides=(1,1),
                         activation=params['activation'],
                         padding='same',))
        model.add(Conv2D(filters=10, kernel_size=params['kernel'], strides=(1,1),
                         activation=params['activation'],
                         padding='same'))
        model.add(Conv2D(filters=10, kernel_size=params['kernel'], strides=(1,1),
                         activation=params['activation'],
                         padding='same'))
        model.add(Flatten())
        model.add(Dropout(rate=params['dropout']))
        model.add(Dense(units=params['hidden_layers'], activation=params['activation']))
        model.add(Dense(nb_out, activation=params['activation']))

        model.compile(loss=params['losses'], optimizer=params['optimizer'],metrics=['mae','acc'])

        start=time.time()
        history = model.fit(seq_array_expanded, label_array, epochs=100, batch_size=params['batch_size'], validation_split=0.1, verbose=2,
                         callbacks = [keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=15, verbose=0, mode='max'),
                                      keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='min', verbose=0)])
                  
        end=time.time()
        run_time=end-start

        return history,model
                                                                            
    scan_object = ta.Scan(seq_array, label_array, model=cnn1d_train, params=p, grid_downsample=0.1)

    return scan_object


    
    
        
    



In [ ]:
seq_array,seq_test_array,label_array,test_df=prep_deeplearn(train_df,test_df,truth_df,sequence_length)

In [97]:
scan_object=minimal()







  0%|          | 0/14 [00:00<?, ?it/s]

Train on 9846 samples, validate on 1095 samples
Epoch 1/100
 - 2s - loss: 6375.4294 - mean_absolute_error: 61.8029 - acc: 0.0067 - val_loss: 3353.5899 - val_mean_absolute_error: 47.1712 - val_acc: 0.0064
Epoch 2/100
 - 1s - loss: 3299.0989 - mean_absolute_error: 46.5291 - acc: 0.0059 - val_loss: 3336.2069 - val_mean_absolute_error: 46.8358 - val_acc: 0.0046
Epoch 3/100
 - 1s - loss: 3049.1879 - mean_absolute_error: 44.0009 - acc: 0.0075 - val_loss: 2187.2907 - val_mean_absolute_error: 34.3823 - val_acc: 0.0100
Epoch 4/100
 - 1s - loss: 1996.6153 - mean_absolute_error: 34.2068 - acc: 0.0100 - val_loss: 1818.9579 - val_mean_absolute_error: 31.6100 - val_acc: 0.0082
Epoch 5/100
 - 1s - loss: 1663.9523 - mean_absolute_error: 31.0858 - acc: 0.0117 - val_loss: 1748.4571 - val_mean_absolute_error: 30.6691 - val_acc: 0.0119
Epoch 6/100
 - 1s - loss: 1633.4552 - mean_absolute_error: 30.5011 - acc: 0.0127 - val_loss: 1549.0139 - val_mean_absolute_error: 29.4801 - val_acc: 0.0137
Epoch 7/100
 - 1







  7%|▋         | 1/14 [00:38<08:16, 38.16s/it]

Train on 9846 samples, validate on 1095 samples
Epoch 1/100
 - 3s - loss: 30.4694 - mean_absolute_error: 31.1503 - acc: 0.0160 - val_loss: 22.4517 - val_mean_absolute_error: 23.1307 - val_acc: 0.0137
Epoch 2/100
 - 3s - loss: 21.1856 - mean_absolute_error: 21.8623 - acc: 0.0196 - val_loss: 19.1387 - val_mean_absolute_error: 19.8102 - val_acc: 0.0201
Epoch 3/100
 - 3s - loss: 19.1119 - mean_absolute_error: 19.7864 - acc: 0.0232 - val_loss: 17.5792 - val_mean_absolute_error: 18.2505 - val_acc: 0.0292
Epoch 4/100
 - 3s - loss: 18.3173 - mean_absolute_error: 18.9878 - acc: 0.0270 - val_loss: 16.0757 - val_mean_absolute_error: 16.7350 - val_acc: 0.0438
Epoch 5/100
 - 3s - loss: 17.7746 - mean_absolute_error: 18.4427 - acc: 0.0321 - val_loss: 17.1846 - val_mean_absolute_error: 17.8521 - val_acc: 0.0292
Epoch 6/100
 - 3s - loss: 17.4542 - mean_absolute_error: 18.1239 - acc: 0.0272 - val_loss: 15.1916 - val_mean_absolute_error: 15.8514 - val_acc: 0.0438
Epoch 7/100
 - 3s - loss: 17.1595 - mean







 14%|█▍        | 2/14 [02:15<11:11, 56.00s/it]

...







 86%|████████▌ | 12/14 [21:44<04:39, 139.95s/it]

Train on 9846 samples, validate on 1095 samples
Epoch 1/100
 - 2s - loss: 9285.5708 - mean_absolute_error: 77.8568 - acc: 0.0066 - val_loss: 7349.3252 - val_mean_absolute_error: 65.9848 - val_acc: 0.0073
Epoch 2/100
 - 1s - loss: 5912.7544 - mean_absolute_error: 58.6832 - acc: 0.0061 - val_loss: 4337.6644 - val_mean_absolute_error: 49.4292 - val_acc: 0.0073
Epoch 3/100
 - 1s - loss: 3746.2234 - mean_absolute_error: 47.5741 - acc: 0.0066 - val_loss: 3373.6891 - val_mean_absolute_error: 46.3824 - val_acc: 0.0046
Epoch 4/100
 - 1s - loss: 3323.7240 - mean_absolute_error: 46.3179 - acc: 0.0049 - val_loss: 3344.4208 - val_mean_absolute_error: 46.9727 - val_acc: 0.0037
Epoch 5/100
 - 1s - loss: 3312.9978 - mean_absolute_error: 46.4251 - acc: 0.0060 - val_loss: 3343.4130 - val_mean_absolute_error: 46.9299 - val_acc: 0.0037
Epoch 6/100
 - 1s - loss: 3312.9648 - mean_absolute_error: 46.3851 - acc: 0.0070 - val_loss: 3342.1685 - val_mean_absolute_error: 46.8436 - val_acc: 0.0046
Epoch 7/100
 - 1







 93%|█████████▎| 13/14 [22:07<01:44, 104.97s/it]

Train on 9846 samples, validate on 1095 samples
Epoch 1/100
 - 2s - loss: 4748.7363 - mean_absolute_error: 53.2382 - acc: 0.0056 - val_loss: 2015.8236 - val_mean_absolute_error: 31.8381 - val_acc: 0.0073
Epoch 2/100
 - 1s - loss: 1544.5216 - mean_absolute_error: 29.4831 - acc: 0.0128 - val_loss: 1484.5902 - val_mean_absolute_error: 29.1318 - val_acc: 0.0110
Epoch 3/100
 - 1s - loss: 1407.5442 - mean_absolute_error: 27.9123 - acc: 0.0152 - val_loss: 1492.4281 - val_mean_absolute_error: 28.2404 - val_acc: 0.0146
Epoch 4/100
 - 1s - loss: 1379.0758 - mean_absolute_error: 27.4990 - acc: 0.0172 - val_loss: 1382.1009 - val_mean_absolute_error: 27.5546 - val_acc: 0.0201
Epoch 5/100
 - 1s - loss: 1303.9067 - mean_absolute_error: 26.4459 - acc: 0.0156 - val_loss: 1328.7662 - val_mean_absolute_error: 26.4914 - val_acc: 0.0201
Epoch 6/100
 - 1s - loss: 1271.1697 - mean_absolute_error: 25.8803 - acc: 0.0159 - val_loss: 1310.4832 - val_mean_absolute_error: 25.7487 - val_acc: 0.0201
Epoch 7/100
 - 1







100%|██████████| 14/14 [23:05<00:00, 90.78s/it] 







TESTING RESULTS

In [ ]:
p=ta.Predict(scan_object)
e=ta.Evaluate(scan_object)

In [ ]:
r=ta.Reporting('030519215433_.csv')


In [101]:
r.best_params()

array([[13.661055597975919, 15.443959118170685, 'logcosh', 'Nadam', 3,
        'tanh', 32, 0],
       [12.94378938805567, 15.045963096928599, 'logcosh', 'Adam', 5,
        'tanh', 32, 1],
       [12.98980499163066, 14.795931387790306, 'logcosh', 'Nadam', 3,
        'tanh', 16, 2],
       [15.14410452472565, 16.938595340766724, 'logcosh', 'Adam', 3,
        'relu', 128, 3],
       [14.235895648067947, 16.23408677222984, 'logcosh', 'Nadam', 5,
        'elu', 16, 4],
       [11.702946186501142, 13.406005870998086, 'mean_squared_error',
        'Adam', 5, 'elu', 32, 5],
       [15.102110302720439, 16.04134943483807, 'logcosh', 'Adam', 10,
        'relu', 16, 6],
       [17.61267311061354, 17.855643389599365, 'logcosh', 'Adam', 10,
        'relu', 512, 7],
       [10.2986853185854, 12.249598861685035, 'mean_squared_error',
        'Adam', 5, 'tanh', 16, 8],
       [16.468773960305132, 17.977037238416067, 'mean_squared_error',
        'Adam', 3, 'tanh', 128, 9]], dtype=object)

In [ ]:
def cnn1d_val(pred,eva,test_df,sequence_length):
    sequence_cols = test_df.columns.difference(['id','cycle','RUL'])
    # We pick the last sequence for each id in the test data
    seq_array_test_last = [[test_df[test_df['id']==id][sequence_cols].values[-sequence_length:]]
                           for id in test_df['id'].unique() if test_df[test_df['id']==id]['cycle'].max()>sequence_length]

    seq_array_test_last = np.concatenate(seq_array_test_last).astype(np.float32)

    smp=seq_array_test_last.shape[0]
    n=seq_array_test_last.shape[2]

    y_mask = [test_df[test_df['id']==id]['cycle'].max()>sequence_length for id in test_df['id'].unique()]
    label_array_test_last = test_df.groupby('id')['RUL'].nth(-1)[y_mask].values
    label_array_test_last = label_array_test_last.reshape(smp,1).astype(np.float32)
    
    scores_test = eva.evaluate(seq_array_test_last, label_array_test_last)
    
    y_pred_test = pred.predict(seq_array_test_last)
    y_true_test = label_array_test_last
    
    sf = scoring_function(y_true_test,y_pred_test)
    rmse_err = rmse(y_true_test,y_pred_test)
    m_err = merror(y_true_test,y_pred_test)
    
    print(m_err,sf,rmse_err)

In [ ]:
def cnn2d_val(pred,eva,test_df,sequence_length):
    sequence_cols = test_df.columns.difference(['id','cycle','RUL'])
    # We pick the last sequence for each id in the test data
    seq_array_test_last = [[test_df[test_df['id']==id][sequence_cols].values[-sequence_length:]]
                           for id in test_df['id'].unique() if test_df[test_df['id']==id]['cycle'].max()>sequence_length]

    seq_array_test_last = np.concatenate(seq_array_test_last).astype(np.float32)

    smp=seq_array_test_last.shape[0]
    n=seq_array_test_last.shape[2]
    
    seq_array_test_last = seq_array_test_last.reshape(smp, sequence_length, n, 1)

    y_mask = [test_df[test_df['id']==id]['cycle'].max()>sequence_length for id in test_df['id'].unique()]
    label_array_test_last = test_df.groupby('id')['RUL'].nth(-1)[y_mask].values
    label_array_test_last = label_array_test_last.reshape(smp,1).astype(np.float32)
    
    scores_test = eva.evaluate(seq_array_test_last, label_array_test_last)
    
    y_pred_test = pred.predict(seq_array_test_last)
    y_true_test = label_array_test_last
    
    sf = scoring_function(y_true_test,y_pred_test)
    rmse_err = rmse(y_true_test,y_pred_test)
    m_err = merror(y_true_test,y_pred_test)
    
    print(m_err,sf,rmse_err)

In [102]:
cnn1d_val(p,e,test_df,sequence_length)

11.423786992612092 [1153.0573] [18.002577]


In [ ]:
print(run_time)

300.3336408138275
